In [26]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
path_code =  "/Users/sanaailla/Desktop/Intern/CODES/"
sys.path.append(path_code)
from Modules import edf
from Modules import desaturation

In [28]:
path_data ="/Users/sanaailla/Desktop/Intern/Data/mars/"
patients = pd.read_csv(path_data + "dir_list.csv",header=None)
patients

,0
0,PA1011
1,PA1089
2,PA1123
3,PA1141
4,PA1150
5,PA1154
6,PA1171
7,PA1178
8,PA1185
9,PA1203


In [29]:
def sat_annot_desats(patient):
    data = edf.verifier_isPSG(path_data,patient)
    annotations = pd.read_csv(path_data + patient + "/Annotations.csv")
    # saturations,frequence = edf.get_signalSat(data,annotations)
    # saturations = edf.filtrer_signalSat(saturations)
    saturations = pd.read_csv(path_data + patient + "/saturations.csv")
    sat_sym = pd.DataFrame(columns=["Heure","SAT"])
    sat_sym["Heure"] = saturations["Heure"]
    baseline = saturations.SAT[0]
    # if baseline<96 :
    #      baseline = (np.mean(saturations.SAT[:-1][(np.abs(np.diff(saturations.SAT)) == 0)]))
    sat_sym["SAT"] = 2*baseline - saturations.SAT
    events =  ["Apnée Centrale","hypopnée Centrale","Apnée Obstructive","Hypopnée"]
    annotations = annotations[annotations["Evènement"].isin(events)][["Heure","Evènement","Durée"]]
    annotations["Heure"] = annotations["Heure"].astype(int)
    annotations["Durée"] = annotations["Durée"].astype(int)
    # desaturations =  desaturation.f_desaturation (saturations, frequence, annotations, 3)
    desaturations = pd.read_csv(path_data + patient + "/desaturations.csv")
    # desatsym = desaturation.f_desaturation (sat_sym, frequence, annotations, 3)
    saturations["Heure"] = saturations["Heure"] .astype("int")
    saturations = saturations.groupby("Heure").mean().reset_index()
    # hypervs =  desatsym[desatsym.MIN_desat < baseline]
    hypervs = pd.read_csv(path_data + patient + "/hypervs.csv")
    desaturations = desaturations[desaturations.MIN_desat < baseline]
    # plt.figure()
    # plt.plot(saturations["SAT"][:4000])
    # plt.axhline(baseline,c="red")
    # plt.show()
    # print(frequence)
    return saturations,desaturations,hypervs,annotations,baseline

In [30]:
saturations_all = pd.DataFrame()
desaturations_all = pd.DataFrame()
hypervs_all = pd.DataFrame()
annotations_all = pd.DataFrame()
baselines_all = pd.DataFrame()
for patient in patients[0]:
    files_path = path_data + patient + "/"
    saturations, desaturations, hypervs ,annotations,baseline = sat_annot_desats(patient)
    saturations['patient'] = patient
    desaturations['patient'] = patient
    hypervs['patient'] = patient
    annotations['patient'] = patient
    saturations_all = pd.concat([saturations_all, saturations])
    desaturations_all = pd.concat([desaturations_all, desaturations])
    hypervs_all = pd.concat([hypervs_all, hypervs])
    baselines_all = pd.concat([baselines_all, pd.DataFrame({"patient": [patient], "baseline": [baseline]})])
    # annotations_all = pd.concat([annotations_all, annotations])
    saturations_file = files_path + "/saturations.csv"
    desaturations_file = files_path + "/desaturations.csv"
    hypervs_file = files_path + "/hypervs.csv"
    #annotations_file = files_path + "/hypervs.csv"
    saturations.drop(columns = "patient").to_csv(saturations_file, index=False)
    desaturations.drop(columns = "patient").to_csv(desaturations_file, index=False)
    hypervs.drop(columns = "patient").to_csv(hypervs_file, index=False)
saturations_all_file = path_data + "saturations_all.csv"
desaturations_all_file = path_data + "desaturations_all.csv"
hypervs_all_file = path_data + "hypervs_all.csv"
# annotations_all_file = path_data + "annotations_all.csv"
saturations_all.to_csv(saturations_all_file, index=False)
desaturations_all.to_csv(desaturations_all_file, index=False)
hypervs_all.to_csv(hypervs_all_file, index=False)
# annotations_all.to_csv(annotations_all_file, index=False)
baselines_all.to_csv(path_data + "baselines.csv", index=False)

Extracting EDF parameters from /Users/sanaailla/Desktop/Intern/Data/mars/PA1011/FE899T1-PA1011V1C1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5900543  =      0.000 ... 23048.996 secs...
Extracting EDF parameters from /Users/sanaailla/Desktop/Intern/Data/mars/PA1089/FE406T1-PA1089V1C1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6643711  =      0.000 ... 25951.996 secs...
Extracting EDF parameters from /Users/sanaailla/Desktop/Intern/Data/mars/PA1123/FE900T1-PA1123V1C1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6683391  =      0.000 ... 26106.996 secs...
Extracting EDF parameters from /Users/sanaailla/Desktop/Intern/Data/mars/PA1141/FE902T1-PA1141V1C1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6744575  =      0.000 ... 26345.996 secs...
Extracting E

In [31]:
def annotate_sig(saturations, annotations):
    events =  ["Apnée Centrale","hypopnée Centrale","Apnée Obstructive","Hypopnée"]
    annotations = annotations[annotations["Evènement"].isin(events)][["Heure","Evènement","Durée"]]
    saturations["Évenement"] = np.NAN
    saturations["Durée"] = np.NAN
    for heure, duree, evenement in zip(annotations.Heure, annotations.Durée, annotations.Evènement):
        ix = saturations[saturations["Heure"] == heure].index.values
        if len(ix)!=0:
            i = ix[0]
            saturations.loc[i:i + duree - 1, "Évenement"] = evenement
            saturations.loc[i:i + duree - 1, "Durée"] = duree
    return saturations

In [32]:
ac,ao,hc,ho = 0,1,2,3

In [33]:
def desaturations_lab(desaturations, saturations,patients, seuil = 90) :
    toreturn = pd.DataFrame(columns = desaturations.columns)
    toreturn["events"] = np.NAN
    for patient in patients:
        desat = desaturations[desaturations["patient"] == patient].reset_index()
        annotations = pd.read_csv(path_data + patient + "/Annotations.csv")
        sats = saturations[saturations["patient"] == patient].reset_index()
        sats = annotate_sig(sats,annotations)
        data = pd.DataFrame(data = desat,columns = desaturations.columns)
        debut_desat = desat["HEURE_debut"].astype(int) + 1
        debut_label = sats.Heure
        events = sats["Évenement"]
        duree_label = sats.Durée
        labels = []
        closest_times = []
        for i in range(len(desat)):
            closest_label = np.NaN
            closest_time = np.NaN
            for label_time,event,duree in zip(debut_label,events,duree_label):
                if label_time < debut_desat[i] and (debut_desat[i] - label_time) < seuil + duree  and label_time not in closest_times:
                    closest_label = event
                    closest_time = label_time
            labels.append(closest_label)
            closest_times.append(closest_time)
        data["events"] = labels
        data = data[~data["events"].isna()].reset_index(drop=True)
        toreturn = pd.concat([data,toreturn])
        print("fin",patient)
    toreturn["events"] =  toreturn["events"].replace({"hypopnée Centrale": hc,"Hypopnée": ho,"Apnée Obstructive": ao,"Apnée Centrale": ac})
    return toreturn

In [34]:
# desaturations_labelise = desaturations_lab(desaturations_all,saturations_all,patients[0])
# desaturations_labelise.to_csv(path_data + "Desaturations_all_labelisee.csv")
desaturations_labelise = pd.read_csv(path_data + "Desaturations_all_labelisee.csv")
desaturations_labelise = desaturations_labelise.drop(desaturations_labelise.columns[0],axis = 1)
desaturations_labelise

,MAX_desat,MIN_desat,MAX_resat,HEURE_debut,HEURE_min,HEURE_fin,DUREE_event,DUREE_desat,DUREE_resat,DELTA_desat,DELTA_resat,PENTE_desat,PENTE_resat,patient,events
0,96.498528,92.498589,95.498543,80447.562500,80473.218750,80498.218750,50.656250,25.656250,25.000000,4.0,2.999954,-0.155907,0.119998,PA1203,2
1,95.498543,92.498589,95.498543,80506.578125,80535.906250,80550.218750,43.640625,29.328125,14.312500,3.0,2.999954,-0.102291,0.209604,PA1203,2
2,95.498543,92.498589,95.498543,80870.875000,80891.546875,80899.546875,28.671875,20.671875,8.000000,3.0,2.999954,-0.145125,0.374994,PA1203,2
3,97.498512,92.498589,96.498528,80913.546875,80943.203125,80966.546875,53.000000,29.656250,23.343750,5.0,3.999939,-0.168599,0.171349,PA1203,2
4,95.498543,92.498589,95.498543,80975.562500,81001.210938,81037.203125,61.640625,25.648438,35.992188,3.0,2.999954,-0.116966,0.083350,PA1203,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,98.498497,90.498619,98.498497,107640.984375,107655.968750,107673.632812,32.648438,14.984375,17.664062,8.0,7.999878,-0.533889,0.452890,PA1011,0
2228,98.498497,91.498604,98.498497,107673.632812,107689.640625,107707.968750,34.335938,16.007812,18.328125,7.0,6.999893,-0.437286,0.381921,PA1011,3
2229,98.498497,90.498619,96.498528,107707.968750,107728.640625,107743.632812,35.664062,20.671875,14.992188,8.0,5.999908,-0.386999,0.400202,PA1011,3
2230,97.498512,90.498619,97.498512,107753.632812,107775.296875,107865.960938,112.328125,21.664062,90.664062,7.0,6.999893,-0.323116,0.077207,PA1011,3


In [35]:
desaturations_labelise.value_counts(subset = "events")

events
0    759
3    627
2    573
1    273
dtype: int64

In [36]:
import numpy as np
from scipy.integrate import trapz
def attribute_hypervs(desaturations,saturations,hypervs,patients,ecart = 0):
    toreturn = pd.DataFrame()
    for patient in patients:
        print(patient)
        desat = desaturations[desaturations["patient"] == patient].reset_index()
        sats = saturations[saturations["patient"] == patient].reset_index()
        hyp = hypervs[hypervs["patient"] == patient].reset_index()
        # print(sats)
        m = baselines_all[baselines_all["patient"] == patient].baseline.values[0]
        i = 0
        ixs = []
        desat["Fin hyp"] = 0
        desat["Hyp"] = 0
        desat["area_up"] = 0
        while i < len(desat):
            seuil = int(desat.DUREE_event[i]) + ecart
            j = 0
            while j < len(hyp):
                dff_time = hyp.HEURE_min[j] - desat.HEURE_debut[i]
                if dff_time >= 0 and dff_time <= seuil:
                    ixs.append((i,j))
                    break
                j+= 1
            desaturation = sats[((sats.Heure >= desat.HEURE_debut[i]) & (sats.Heure <= desat.HEURE_fin[i]))].SAT
            x = desaturation.index.values
            y = desaturation.values
            baseline = [m for _ in range(len(desaturation))]
            desat.loc[i, "area_down"] = trapz(np.minimum(y, baseline) - baseline, x)
            if (desat.MIN_desat[i] >= m):
                plt.plot(desaturation)
                plt.axhline(m,c="red")
                plt.show()
            i+= 1
        for (d,h) in ixs:
            signal_hyp = sats[(sats.Heure >= hyp.HEURE_min[h]) & (sats.Heure <= hyp.HEURE_fin[h])].SAT
            indice_sig = (signal_hyp >= m)
            indice_descente = indice_sig[indice_sig == False].index
            if (len(indice_descente)!=0):
                indice_descente = indice_descente.values[0]
                heure_fin = sats.iloc[indice_descente].Heure
            else :
                heure_fin = hyp.HEURE_fin[h]
            desaturation = sats[(sats.Heure >= desat.HEURE_debut[d]) & (sats.Heure <= heure_fin)].SAT
            hypo = sats[(sats.Heure >= desat.HEURE_min[d]) & (sats.Heure <= heure_fin)].SAT
            if desaturation.values[-1] <m:
                hypo = hypo[:-1]
            x = hypo.index.values
            y = hypo.values
            reference_line = [m for _ in range(len(hypo))]
            desat.loc[d, "area_up"] = trapz(np.maximum(y, reference_line) - reference_line, x)
            #desat.loc[d, "area_down"] = trapz(np.maximum(y, reference_line) - reference_line, x)
            desat.loc[d,"Fin hyp"] = heure_fin
            desat.loc[d,"Hyp"] = 1
        toreturn = pd.concat([desat,toreturn])
        print("fin", patient)
    return toreturn[toreturn.area_down>0]

In [37]:
final_desats = attribute_hypervs(desaturations_labelise,saturations_all,hypervs_all,patients[0],0)
final_desats.columns
# saturations_all[saturations_all["patient"] == patients[0][1]]

PA1011
fin PA1011
PA1089
fin PA1089
PA1123
fin PA1123
PA1141
fin PA1141
PA1150
fin PA1150
PA1154
fin PA1154
PA1171
fin PA1171
PA1178
fin PA1178
PA1185
fin PA1185
PA1203
fin PA1203


Index(['index', 'MAX_desat', 'MIN_desat', 'MAX_resat', 'HEURE_debut',
       'HEURE_min', 'HEURE_fin', 'DUREE_event', 'DUREE_desat', 'DUREE_resat',
       'DELTA_desat', 'DELTA_resat', 'PENTE_desat', 'PENTE_resat', 'patient',
       'events', 'Fin hyp', 'Hyp', 'area_up', 'area_down'],
      dtype='object')

In [38]:
features = ["patient","MAX_desat","MIN_desat","MAX_resat","DUREE_event", "DUREE_desat","DUREE_resat","DELTA_desat","DELTA_resat","PENTE_desat","PENTE_resat" ,"area_down","area_up","events"]
final_desats = final_desats[features]
final_desats

,patient,MAX_desat,MIN_desat,MAX_resat,DUREE_event,DUREE_desat,DUREE_resat,DELTA_desat,DELTA_resat,PENTE_desat,PENTE_resat,area_down,area_up,events


In [39]:
# final_desats.to_csv(path_data + "final_desats.csv",index = False)
final_desats = pd.read_csv(path_data + "final_desats.csv")
final_desats = final_desats[features]
final_desats.to_csv(path_data + "final_desats.csv",index = False)

In [40]:
def display_desaturations(desaturations,saturations,patient):
    num_plots = len(desaturations[desaturations.Hyp == 1])
    num_columns = 3
    num_rows = (num_plots + num_columns - 1) // num_columns
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5*num_rows))
    j = 0
    for i in range(len(desaturations)):
        heure_fin = desaturations.HEURE_fin[i]
        if  desaturations["Fin hyp"][i] !=0:
            row = j // num_columns
            col = j % num_columns
            ax = axes[row, col]
            heure_fin = desaturations["Fin hyp"][i]
            # heure_fin = desaturations.HEURE_fin
            desaturation = saturations[(saturations.Heure >= desaturations.HEURE_debut[i]) & (saturations.Heure <= (desaturations.HEURE_fin[i] + 1))].SAT
            ax.plot(desaturation,c = "blue")
            m = baselines_all[baselines_all["patient"] == patient].baseline.values[0]
            ax.plot(saturations[(saturations.Heure >= desaturations.HEURE_min[i]) & (saturations.Heure <= (heure_fin + 1)) & (saturations.SAT >= m)].SAT,c = "green")
            ax.axhline(m,c="red")
            j+=1
    plt.tight_layout()
    plt.show()

In [43]:
desat = pd.read_csv("statistiques.csv")
summary = desat.describe()
summary


,MAX_desat,MIN_desat,MAX_resat,DUREE_event,DUREE_desat,DUREE_resat,DELTA_desat,DELTA_resat,PENTE_desat,PENTE_resat,area_down,area_up,DD100,DDmax,DDbase,TRatio,PRatio
count,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000,2232.000000
mean,97.964007,92.174669,97.991784,49.532027,19.468354,30.063673,5.789427,5.817116,-3.353351,0.323195,-130.521752,11.450891,7.825331,5.789338,5.132428,1.143694,8.229758
std,1.284537,2.690547,1.306774,20.382729,8.317665,18.676972,2.750252,2.834195,41.659935,0.420097,113.814272,27.152401,2.690547,2.750210,3.333847,2.584693,88.859959
min,92.498589,77.498817,92.498589,10.007812,0.007812,0.992188,3.000000,2.999954,-896.000000,0.028214,-1738.393717,0.000000,3.501472,2.999954,-1.999968,0.000398,0.015147
25%,97.498512,90.498619,97.498512,35.656250,13.351562,17.333984,4.000000,3.999939,-0.434183,0.116929,-173.806657,0.000000,5.501442,3.999939,2.999957,0.439541,0.911687
50%,98.498497,92.498589,98.498497,47.363281,20.007812,27.000000,5.000000,4.999924,-0.292446,0.203373,-102.081492,0.000000,7.501411,4.999924,4.999922,0.709730,1.419879
75%,98.498497,94.498558,98.498497,59.984375,26.007812,37.656250,7.000000,6.999893,-0.202252,0.386993,-53.188259,8.024583,9.501381,6.999893,6.999891,1.078760,2.332400
max,99.498482,96.498528,99.498482,119.992188,115.664062,108.328125,18.000000,21.999664,-0.051874,5.039293,-0.401570,209.391440,22.501183,17.999725,16.999744,73.674419,2095.865314
